# Data Platform Team Co-op Case Study
## By Yujia Zheng, email: y326zhen@uwaterloo.ca

In [1]:
# Import useful packages.
import pandas as pd
import numpy as np

In [2]:
# The given stringified table.
data = 'Airline Code;DelayTimes;FlightCodes;To_From\nAir Canada (!);[21, 40];20015.0;WAterLoo_NEWYork\n<Air France> (12);[];;Montreal_TORONTO\n(Porter Airways. );[60, 22, 87];20035.0;CALgary_Ottawa\n12. Air France;[78, 66];;Ottawa_VANcouvER\n""".\\.Lufthansa.\\.""";[12, 33];20055.0;london_MONTreal\n'

In [3]:
# Split the string data by \n first to get each row of the table.
# Then, split by ; to get each cell.
data_lists = [sub.split(';') for sub in data.split('\n')]
# Remove the last list which only contains the empty string.
data_lists.pop()
# Take a look at the coverted lists of data.
data_lists

[['Airline Code', 'DelayTimes', 'FlightCodes', 'To_From'],
 ['Air Canada (!)', '[21, 40]', '20015.0', 'WAterLoo_NEWYork'],
 ['<Air France> (12)', '[]', '', 'Montreal_TORONTO'],
 ['(Porter Airways. )', '[60, 22, 87]', '20035.0', 'CALgary_Ottawa'],
 ['12. Air France', '[78, 66]', '', 'Ottawa_VANcouvER'],
 ['""".\\.Lufthansa.\\."""', '[12, 33]', '20055.0', 'london_MONTreal']]

In [4]:
# Convert the data_lists into a pandas dataframe.
# Use the first sublist as the column names of the dataframe.
df = pd.DataFrame(data_lists[1:], columns = data_lists[0])
# Take a look at the initial dataframe.
df

,Airline Code,DelayTimes,FlightCodes,To_From
0,Air Canada (!),"[21, 40]",20015.0,WAterLoo_NEWYork
1,<Air France> (12),[],,Montreal_TORONTO
2,(Porter Airways. ),"[60, 22, 87]",20035.0,CALgary_Ottawa
3,12. Air France,"[78, 66]",,Ottawa_VANcouvER
4,""""""".\.Lufthansa.\.""""""","[12, 33]",20055.0,london_MONTreal


## Requirement 1

In [5]:
FlightCodes = list(df['FlightCodes'])

# Find the index of the first non-missing flight code.
valid_idx = 0
for i in range(len(FlightCodes)):
    if FlightCodes[i] != '':
        valid_idx = i
        break

# Using the fact that flight codes are supposed to increase by 10 with each row, 
# calculate the first and last flight code in the data,
# and use range() to fill in all the flight codes in the middle.
first_code = int(float(FlightCodes[valid_idx])) - valid_idx*10
last_code = int(float(FlightCodes[valid_idx])) + (len(FlightCodes)-1-valid_idx)*10
all_codes = list(range(first_code, last_code+10, 10))

# Assume in the given data, there is at least one flight code that is non-missing,
# e.g. ['', '', '20035.0', '', '']
# The code above could fill all the missing codes.
df['FlightCodes'] = all_codes
# Take a look at the dataframe.
df

,Airline Code,DelayTimes,FlightCodes,To_From
0,Air Canada (!),"[21, 40]",20015,WAterLoo_NEWYork
1,<Air France> (12),[],20025,Montreal_TORONTO
2,(Porter Airways. ),"[60, 22, 87]",20035,CALgary_Ottawa
3,12. Air France,"[78, 66]",20045,Ottawa_VANcouvER
4,""""""".\.Lufthansa.\.""""""","[12, 33]",20055,london_MONTreal


In [6]:
# Check that the type of the FlightCodes column is int.
df['FlightCodes'].dtype

dtype('int64')

## Requirement 2

In [7]:
# Split To_From into 2 separate columns and convert all to upper case.
df[['To', 'From']] = df['To_From'].apply(lambda x: pd.Series(str.upper(str(x)).split('_')))
# Drop the original To_From column.
df = df.drop(['To_From'], axis=1)
# Take a look at the dataframe.
df

,Airline Code,DelayTimes,FlightCodes,To,From
0,Air Canada (!),"[21, 40]",20015,WATERLOO,NEWYORK
1,<Air France> (12),[],20025,MONTREAL,TORONTO
2,(Porter Airways. ),"[60, 22, 87]",20035,CALGARY,OTTAWA
3,12. Air France,"[78, 66]",20045,OTTAWA,VANCOUVER
4,""""""".\.Lufthansa.\.""""""","[12, 33]",20055,LONDON,MONTREAL


## Requirement 3

In [8]:
# Clean the Airline Codes to have no punctuation except spaces in the middle.
# Keep only alphabetical letters and spaces in the middle, remove all puctuations and digits.
# The regex expression means: 
# replace any character that is not a space or alphabetical letters (upper or lower cases) with nothing.
# Since we only keep the spaces in the middle, we need to strip the spaces at the beginning and end.
df['Airline Code'] = df['Airline Code'].str.replace('[^ A-Za-z]+','').str.strip(' ')
# Take a look at the final dataframe.
df

/var/folders/_0/jx49pw6d44l_9fx2ktgfwxlc0000gn/T/ipykernel_91840/4228773063.py:6: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Airline Code'] = df['Airline Code'].str.replace('[^ A-Za-z]+','').str.strip(' ')


,Airline Code,DelayTimes,FlightCodes,To,From
0,Air Canada,"[21, 40]",20015,WATERLOO,NEWYORK
1,Air France,[],20025,MONTREAL,TORONTO
2,Porter Airways,"[60, 22, 87]",20035,CALGARY,OTTAWA
3,Air France,"[78, 66]",20045,OTTAWA,VANCOUVER
4,Lufthansa,"[12, 33]",20055,LONDON,MONTREAL


In [9]:
# Generate a stringified version of the cleaned final table, if needed.
stringified_table = df.to_csv(sep=';', line_terminator='\n', index=False)
stringified_table

'Airline Code;DelayTimes;FlightCodes;To;From\nAir Canada;[21, 40];20015;WATERLOO;NEWYORK\nAir France;[];20025;MONTREAL;TORONTO\nPorter Airways;[60, 22, 87];20035;CALGARY;OTTAWA\nAir France;[78, 66];20045;OTTAWA;VANCOUVER\nLufthansa;[12, 33];20055;LONDON;MONTREAL\n'